In [ ]:
import openslide
from PIL import Image
from pathlib import Path
import tensorflow as tf
import numpy as np
import pandas
import cv2
import os

In [ ]:
def get_slide_mpp(slide_path):
    """Get the microns per pixel (resolution)"""
    slide = openslide.OpenSlide(slide_path)
    
    # Try to get MPP directly
    try:
        mpp_x = float(slide.properties[openslide.PROPERTY_NAME_MPP_X])
        mpp_y = float(slide.properties[openslide.PROPERTY_NAME_MPP_Y])
        slide.close()
        return mpp_x, mpp_y, None
    except (KeyError, ValueError):
        pass
    
    # Try to calculate from resolution
    try:
        spacing_x = float(slide.properties['tiff.XResolution'])
        spacing_y = float(slide.properties['tiff.YResolution'])
        unit = slide.properties.get('tiff.ResolutionUnit')
        
        # Convert based on unit (2 = inch, 3 = centimeter)
        if unit == '2':  # inches
            mpp_x = 25400 / spacing_x  # 25400 microns per inch
            mpp_y = 25400 / spacing_y
        elif unit == '3':  # centimeters
            mpp_x = 10000 / spacing_x  # 10000 microns per centimeter
            mpp_y = 10000 / spacing_y
            
        slide.close()
        return mpp_x, mpp_y, None
    except (KeyError, ValueError):
        pass
    
    # Get available properties if standard methods fail
    properties = dict(slide.properties)
    slide.close()
    
    # Look for any property that might contain resolution info
    mpp_candidates = {k: v for k, v in properties.items() 
                     if 'resolution' in k.lower() or 'mpp' in k.lower() or 'pixel' in k.lower()}
    
    return None, None, mpp_candidates  # Return None and candidates for manual inspection

class UniversalWSI():
    def __init__(self, wsi_path):
        self.wsi_path = wsi_path
        self.ext = Path(wsi_path).suffix
        if self.ext == '.ndpi':
            self.wsi = openslide.OpenSlide(wsi_path)
        elif self.ext == '.svs':
            self.wsi = openslide.OpenSlide(wsi_path)
        elif self.ext == '.dcm':
            self.wsi = openslide.OpenSlide(wsi_path)
        else:
            raise ValueError(f"Unsupported file extension: {self.ext}")
    
    def get_level_0_dimensions(self):
        if self.ext == '.ndpi':
            width, height = self.wsi.level_dimensions[0]
        elif self.ext == '.svs':
            width, height = self.wsi.level_dimensions[0]
        elif self.ext == '.dcm':
            width, height = self.wsi.dimensions
        
        return width, height
    
    def get_level_0_mpp(self):
        """Get the microns per pixel (resolution)"""
        mpp_x, mpp_y, mpp_candidates = get_slide_mpp(self.wsi_path)
        
        if mpp_x is not None and mpp_y is not None:
            assert round(mpp_x, 3) == round(mpp_y, 3), f"MPP x {mpp_x} and y {mpp_y} are not equal for {self.wsi_path}"
            return mpp_x
        else:
            raise ValueError(f"No MPP found for {self.wsi_path}, check out alternative output {mpp_candidates}")
    
    def read_ground_region(self, TL, tile_shape):
        if self.ext == '.ndpi':
            image = self.wsi.read_region(TL, 0, tile_shape)
            if image.mode == 'RGBA':
                image = image.convert('RGB')
            return image
        elif self.ext == '.svs':
            image = self.wsi.read_region(TL, 0, tile_shape)
            if image.mode == 'RGBA':
                image = image.convert('RGB')
            return image
        elif self.ext == '.dcm':
            image = self.wsi.read_region(TL, 0, tile_shape)
            if image.mode == 'RGBA':
                image = image.convert('RGB')
            return image

In [ ]:
slide_pth = "slides/1022220222159016_5.svs"

slide = UniversalWSI(slide_pth)
reg_slide = openslide.open_slide(slide_pth)

In [ ]:
def generate_tile_coordinates(wsi_path, tile_size=1000):
    """
    Generates a 2D list of (x, y) coordinates for tiling the whole slide image.
    
    Args:
        wsi_path (str): Path to the whole slide image (.svs, .ndpi, etc.).
        tile_size (int): Size of each tile in pixels (default 1000x1000).
    
    Returns:
        list: A 2D list containing coordinate tuples (x, y).
    """
    # Open the slide and get dimensions
    slide = UniversalWSI(wsi_path)
    width, height = slide.get_level_0_dimensions()  # Get level 0 dimensions
    # slide.close()  # Close to free memory

    # Generate coordinates
    coordinates = [
        (x, y) 
        for y in range(0, height, tile_size) 
        for x in range(0, width, tile_size)
    ]

    return coordinates

# Example usage
svs_file_path = slide_pth
tile_coordinates = generate_tile_coordinates(svs_file_path)

# Print first few coordinates
print(tile_coordinates)

In [ ]:
model = tf.keras.models.load_model('region_classifier_1.h5')

In [ ]:
tile_shapes = (1000,1000)
cur_img = slide.read_ground_region(TL=(0,0), tile_shape=tile_shapes)

print(cur_img)

img_array = tf.expand_dims(cur_img, 0)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

conf_values = list(score.numpy())

print(conf_values)

print(np.argmax(score))

In [ ]:
slide.get_level_0_dimensions # UniversalWSI Object

w0, h0 = reg_slide.level_dimensions[0] # regular openslide
print(w0, h0)

reg_slide.get_thumbnail((int(w0/16),int(h0/16)))

In [ ]:
tile_shapes = (1000,1000)

adq_threshold = 0.8

pred_data = []
for coord in tile_coordinates:
    cur_img = slide.read_ground_region(TL=coord, tile_shape=tile_shapes)
    img_array = tf.expand_dims(cur_img, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    class_pred = np.argmax(score)
    pred_data.append(class_pred)

dataframe = pandas.DataFrame({
    "Coordinates":tile_coordinates,
    "Predictions": pred_data
})

dataframe.to_csv("ucsf_1_tile_predictions.csv", index=False)

In [ ]:
# USE IF THRESHOLD HIGHER FOR ADEQUATE

tile_shapes = (1000,1000)

adq_threshold = 0.8

pred_data = []
for coord in tile_coordinates:
    cur_img = slide.read_ground_region(TL=coord, tile_shape=tile_shapes)
    img_array = tf.expand_dims(cur_img, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    top_idx = np.argmax(score)
    top_score = score[top_idx]

    # custom threshold for adequate

    if(top_idx == 0 and top_score < adq_threshold):
        second_idx = np.argsort(score)[-2]  
        top_idx = second_idx 

    pred_data.append(top_idx)

dataframe = pandas.DataFrame({
    "Coordinates":tile_coordinates,
    "Predictions": pred_data
})

dataframe.to_csv("0.8_msk_1_tile_predictions.csv", index=False)

In [ ]:
extracted_dataframe = pandas.read_csv("ucsf_1_tile_predictions.csv")

# for index, row in extracted_dataframe.iterrows():
#     print(f"Index: {index}, Row: {row['Coordinates'][0]}, {row['Predictions']}")

In [ ]:
def save_confidence_heatmap_multi(data, topview_img_pil, save_dir):
        topview_img_cv = cv2.imread(topview_img_pil)
        topview_img_cv = cv2.cvtColor(topview_img_cv, cv2.COLOR_RGB2BGR)

        heatmap = np.zeros((*topview_img_cv.shape[:2], 3), dtype=np.uint8)
        print(topview_img_cv.shape[:2])

        # Iterate through the patches
        idx = 0
        for index, row in data.iterrows():
            # Extract the bounding box and confidence score
            TL_x, TL_y = tile_coordinates[idx]
            BR_x = TL_x + tile_shapes[0]
            BR_y = TL_y + tile_shapes[1]
            
            topview_downsampling_factor = 16
            # Adjust the coordinates for the downsampling factor
            TL_x_adj = int(TL_x / topview_downsampling_factor)
            TL_y_adj = int(TL_y / topview_downsampling_factor)
            BR_x_adj = int(BR_x / topview_downsampling_factor)
            BR_y_adj = int(BR_y / topview_downsampling_factor)

            if(row["Predictions"]) == 0: #BGR
                  color = [0, 255, 0]
            elif(row["Predictions"]) == 1:
                  color = [0, 0, 255]
            elif(row["Predictions"]) == 2:
                  color = [255, 0 ,0]

            heatmap[TL_y_adj:BR_y_adj, TL_x_adj:BR_x_adj] = color

            idx += 1

        heatmap_colored = heatmap

        overlay_img_cv = cv2.addWeighted(topview_img_cv, 0.7, heatmap_colored, 0.3, 0)

        overlay_img_pil = Image.fromarray(
            cv2.cvtColor(overlay_img_cv, cv2.COLOR_BGR2RGB)
        )

        overlay_img_pil.save(os.path.join(save_dir, "region_visual_result.png"))

def save_confidence_heatmap(data, topview_path, save_dir):

    # ------------------------------------------------------------------ load
    topview_img_cv = cv2.imread(topview_path)            # BGR in, stays BGR
    H, W = topview_img_cv.shape[:2]

    heatmap = np.zeros((H, W, 3), dtype=np.uint8)        # colour canvas
    mask    = np.zeros((H, W),    dtype=np.uint8)        # 1 where coloured

    idx = 0
    for _, row in data.iterrows():
        TL_x, TL_y = tile_coordinates[idx]
        BR_x = TL_x + tile_shapes[0]
        BR_y = TL_y + tile_shapes[1]

        ds = 16                                          # down-sampling
        TL_x //= ds; TL_y //= ds; BR_x //= ds; BR_y //= ds

        if row["Predictions"] == 0:                      # ONLY class 0
            heatmap[TL_y:BR_y, TL_x:BR_x] = (  0,255,  0)  # green, BGR
            mask[   TL_y:BR_y, TL_x:BR_x] = 1

        idx += 1

    # ---------------------------------------------------------------- blend
    # Make a version that has the green tint everywhere (for convenience)
    blended = cv2.addWeighted(topview_img_cv, 0.7, heatmap, 0.3, 0)

    # Now copy pixels from `blended` ONLY where mask == 1
    overlay_img_cv = topview_img_cv.copy()
    overlay_img_cv[mask.astype(bool)] = blended[mask.astype(bool)]

    # ---------------------------------------------------------------- save
    overlay_img_pil = Image.fromarray(cv2.cvtColor(overlay_img_cv,
                                                   cv2.COLOR_BGR2RGB))
    overlay_img_pil.save(os.path.join(save_dir, "latest_region_visual_result.png"))

In [ ]:
save_confidence_heatmap(extracted_dataframe, "thumbnails/thumbnail_1.png", "results")